# SIADS 516: Homework 3
Version 2.0.20201020.1
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

This homework assignment builds on the Spark DataFrame material we covered in class.
You will be using a compressed version of the Yelp Academic Dataset.  The data set is provided for you in the data/yelp-academic sub-folder of this notebook's directory and you should not need to download it again if you're working on the Coursera hosted notebook environment.

You might want to refer to the lecture companion notebooks (in workspace-files/resources/lecture_notebooks or equivalently via Coursera as "Ungraded Lab: Spark Core Demo" and "Ungraded Lab: Spark SQL Demo) for hints about libraries to import, how to set up a SparkSession, and how to read data files.

You will notice that there are a **lot** of reviews.  You might want to work off a small sample (i.e. use the sample() function in Spark) to work on a reduced size dataset while you're developing your solution.

**You should take care to document your work, preferably using markdown blocks. In-code commenting is also 
a good idea.**

In [10]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

### <font color="magenta">Q1: How many users have received more than 5000 cool votes?</font>

In [11]:
# insert your code here
spark = SparkSession.builder.master("local[*]").appName("Homework3").getOrCreate()
user = spark.read.json("../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_user.json.gz")
user.filter(user["compliment_cool"] > 5000).count()

79

There are 79 users have received more than 5,000 cool votes.

### <font color="magenta">Q2: What are the top 10 most useful positive reviews?</font>

In [12]:
# insert your code here
review = spark.read.json("../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_review.json.gz")
positive_review = review.filter(review["stars"] >= 4)
positive_review.sort("useful", ascending = False).select("business_id", "useful", "stars").take(10)

[Row(business_id='t-o_Sraneime4DDhWrQRBA', useful=358, stars=5.0),
 Row(business_id='6Suj9mb9565xjAKHMXJLmw', useful=278, stars=5.0),
 Row(business_id='JLbgvGM4FXh9zNP4O5ZWjQ', useful=244, stars=4.0),
 Row(business_id='IapQwLdAwztQYN99pQui3w', useful=241, stars=5.0),
 Row(business_id='DkYS3arLOhA8si5uUEmHOw', useful=235, stars=4.0),
 Row(business_id='_5PJ4GHIXNdUdXtohylKGQ', useful=229, stars=4.0),
 Row(business_id='Cf0iV72DTqR0ggBje2d0sg', useful=218, stars=4.0),
 Row(business_id='u4sTiCzVeIHZY8OlaL346Q', useful=218, stars=4.0),
 Row(business_id='77h11eWv6HKJAgojLx8G4w', useful=215, stars=4.0),
 Row(business_id='Ka00H3EHLLiPNpMj1V4Hcw', useful=215, stars=5.0)]

Insert your interpretation here.

### <font color="magenta">Q3: During what hour of the day do most checkins occur?</font>

In [14]:
# insert your code here
checkin = spark.read.json("../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_checkin.json.gz")
checkin.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)



In [16]:
checkin.select("date").show(5)

+--------------------+
|                date|
+--------------------+
|2016-04-26 19:49:...|
|2011-06-04 18:22:...|
|2014-12-29 19:25:...|
| 2016-07-08 16:43:30|
|2010-06-26 17:39:...|
+--------------------+
only showing top 5 rows



In [17]:
checkin_split_date = checkin.withColumn("date_only", F.split(F.col("date"), " ").getItem(0)).withColumn("time_only", F.split(F.col("date"), " ").getItem(1))

In [19]:
checkin_split_date.show(5)

+--------------------+--------------------+----------+---------+
|         business_id|                date| date_only|time_only|
+--------------------+--------------------+----------+---------+
|--1UhMGODdWsrMast...|2016-04-26 19:49:...|2016-04-26|19:49:16,|
|--6MefnULPED_I942...|2011-06-04 18:22:...|2011-06-04|18:22:23,|
|--7zmmkVg-IMGaXbu...|2014-12-29 19:25:...|2014-12-29|19:25:50,|
|--8LPVSo5i0Oo61X0...| 2016-07-08 16:43:30|2016-07-08| 16:43:30|
|--9QQLMTbFzLJ_oT-...|2010-06-26 17:39:...|2010-06-26|17:39:07,|
+--------------------+--------------------+----------+---------+
only showing top 5 rows



In [20]:
# Remove "date_only" column
checkin_split_date = checkin_split_date.select("time_only", F.regexp_replace(F.col("time_only"), "[\$#,]", "").alias("time"))
checkin_split_date.show(5)

+---------+--------+
|time_only|    time|
+---------+--------+
|19:49:16,|19:49:16|
|18:22:23,|18:22:23|
|19:25:50,|19:25:50|
| 16:43:30|16:43:30|
|17:39:07,|17:39:07|
+---------+--------+
only showing top 5 rows



In [21]:
checkin_split_date = checkin_split_date.withColumn("time_only", F.hour("time"))
checkin_split_date.show(5)

+---------+--------+
|time_only|    time|
+---------+--------+
|       19|19:49:16|
|       18|18:22:23|
|       19|19:25:50|
|       16|16:43:30|
|       17|17:39:07|
+---------+--------+
only showing top 5 rows



In [22]:
checkin_split_date.groupBy("time_only").count().sort("count", ascending = False).show(5)

+---------+-----+
|time_only|count|
+---------+-----+
|       19|13481|
|       23|13207|
|       22|13191|
|       18|13177|
|       21|12960|
+---------+-----+
only showing top 5 rows



19:00 is the most common time that customers are checking in to business.

### <font color="magenta">Q4: Sentiment analysis</font>

a. List the 50 most common non-stopword words that are unique to *positive* reviews.

b. List the 50 most common non-stopword words that are unique to *negative* reviews.

You can use the stopword list that was introduced in the lecture materials or you can 
find/devise one of your own.

You will need to define what constitutes a positive review and what constitutes a negative review.  We highly recommend that you use the number of stars to figure this out.  Be sure to provide a rationale for your choice

As an example, consider the following two reviews:

* Positive: The meal was great, and the service was the best we ever experienced.
* Negative: The meal was awful.  It was the worst thing we ever experienced.

Assume our stopwords are {'the','was','and','the','was','we','it'}

* Positive unique: {'great', 'service', 'best'}

* Negative unique: {'awful', 'worst', 'thing'}

In this example, each unique word occurs just once, so the concept of "top 50" doesn't make sense.  For your data, you'll need to count the number of times each unique word occurs.

In [34]:
# stopwords are from nltk
# import nltk
# from nltk.corpus import stopwords
# print(stopwords.words('english'))
stopwords_lower = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
stopwords_upper = [word.title() for word in stopwords_lower]

In [24]:
import re

In [25]:
# insert your code here
negative = review.filter(review["stars"] <= 2)

positive_count = positive_review.withColumn("word", F.explode(F.split(F.col("text"), " "))).groupBy("word").count().sort("count", ascending = False)
positive_count.show(5)

+----+--------+
|word|   count|
+----+--------+
| the|17081648|
| and|15682454|
|   a|10814177|
|   I| 9855342|
|  to| 9514746|
+----+--------+
only showing top 5 rows



In [26]:
negative_count = negative.withColumn("word", F.explode(F.split(F.col("text"), " "))).groupBy("word").count().sort("count", ascending = False)
negative_count.show(5)       

+----+-------+
|word|  count|
+----+-------+
| the|8977366|
| and|6548915|
|  to|6265941|
|   I|5927100|
|   a|4843924|
+----+-------+
only showing top 5 rows



In [35]:
# Filter out the stopwords
positive_count = positive_count.filter(~F.col("word").isin(stopwords_lower))
positive_count = positive_count.filter(~F.col("word").isin(stopwords_upper))
negative_count = negative_count.filter(~F.col("word").isin(stopwords_lower))
negative_count = negative_count.filter(~F.col("word").isin(stopwords_upper))

In [38]:
positive_count.show(51)

+----------+-------+
|      word|  count|
+----------+-------+
|          |6764732|
|     place|1605893|
|     great|1451596|
|      food|1299393|
|      good|1276963|
|      like|1182229|
|       get|1129261|
|    really| 974964|
|      time| 974529|
|       one| 953405|
|     would| 907892|
|        go| 855561|
|      also| 807717|
|   service| 803057|
|      back| 799836|
|    always| 773386|
|      best| 688929|
|         -| 679102|
|      I've| 670023|
|       got| 658937|
|      love| 639154|
|definitely| 620542|
|      nice| 602586|
|    little| 596287|
|       I'm| 592725|
|        us| 552984|
|      even| 551470|
|     staff| 531615|
| recommend| 519607|
|      come| 515932|
|       try| 507643|
|      made| 502427|
|  friendly| 486263|
|      came| 479101|
|   ordered| 472642|
|     first| 467846|
|      make| 467165|
|     Great| 445766|
|      much| 440341|
|         &| 439775|
|     could| 428251|
|       it.| 414699|
|      went| 414570|
|    pretty| 409164|
|      well| 

In [39]:
negative_count.show(51)

+--------+-------+
|    word|  count|
+--------+-------+
|        |3302776|
|   would| 722748|
|     get| 717371|
|    like| 654595|
|    food| 601845|
|     one| 582545|
|   place| 560382|
|    time| 511689|
|    back| 495797|
|    even| 467004|
|      go| 451549|
| service| 449509|
|    said| 438981|
|    told| 438074|
|      us| 437888|
|     got| 409855|
|   never| 383058|
|    good| 362548|
|   could| 356440|
|   asked| 348414|
|    came| 334808|
|    went| 307779|
|  really| 305235|
| ordered| 294755|
|   order| 294234|
|     I'm| 282640|
|   going| 280400|
|  people| 270699|
| minutes| 268319|
|    know| 257905|
|       -| 253441|
|    come| 250879|
| another| 248271|
|    took| 241361|
|    give| 237340|
|     it.| 235939|
|    take| 234470|
|    make| 233882|
|       2| 230535|
|     two| 229880|
|   first| 227559|
|customer| 222354|
|   still| 221491|
|  called| 218973|
|    much| 217976|
|    want| 217446|
|    also| 205934|
|    I've| 205285|
|     way| 196840|
|    made| 1

Insert your interpretation here.